# Card Game Simulation Analysis - Pandas

### Background:
Utilizing pandas to store and analyze the results of simulated gameplay (In previous repository).

## Question 1

In [61]:
#importing numpy
import numpy as np
# Initialize deck
deck = list(range(1, 14)) * 4

def add(deck):
    s=0
    for x in deck:
        s=s+x
    return s

# Define function to deal cards to two players
def deal():
    # Initialize player hands
    p1 = []
    p2 = []
    
    # Initialize and shuffle deck
    deck = list(range(1, 14)) * 4
    np.random.shuffle(deck)
    
    # Deal cards to each player
    for i in range(len(deck)):
        if i % 2:
            p2.append(deck.pop(0))
        else:
            p1.append(deck.pop(0))
            
    return p1, p2

### Steps 1.1:
Add the capability to track the number of ties for each game. In this total, you should include not only the ties that occur at the beginning of a round, but any additional ties that occur during a tie breaker.

### Step 1.2: 
Add the capability to track the largest total won during a round in a given game, which could occur in a standard round (with no ties) or a tiebreaker round (with one or more ties). You do not need to distinguish whether the total was won in a standard or tiebreaker round.

### Step 1.3: 
Add the capability to play multiple games within the function, using a new argument games (equals 1 by default). The function should track the following statistics for each game, stored in a dictionary (ideally as separate entries, which will facilitate next steps):

Initial totals (i.e., sum of card values for initially dealt hands) for player 1 and player 2, respectively
Number of ties during gameplay
Maximum round total won (i.e., maximum sum of card values won during a round, potentially including tiebreakers)
Final totals (i.e., sum of card values after completing gameplay) for player 1 and player 2, respectively

In [139]:
#defining play
#Step 1.1 - Adding 'tie' to track the number of ties during a game
#Step 1.2 - Adding 'max_claim' to find the largest toal won during game
#Step 1.3 - Adding argument 'games' to facilitate multiple games 
#Step 1.4 - Dictionary 'results' to collect statistics of game 
# results_l to store statistics of results of all games
def play(games=1):
    global results_l
    results_l = []
    game = 1
    
    while game <= games:
        # Deal cards to players, initialize claim lists
        p1, p2 = deal()
        s1=sum(p1)
        s2=sum(p2)
        p1_claim, p2_claim, claim = [], [], []
        tie=0

        # Collect statistics
        S = {}

        # Calculate initial statistics
        S['initial_sums'] = (sum(p1), sum(p2))

        # Play!
        k = 1 # round counter
        i = 13 # maximum number of rounds
        while i > 0:
            # Print round
            #print('Round:', k)

            # Each player plays two cards
            p1_play = [p1.pop(0), p1.pop(0)]
            p2_play = [p2.pop(0), p2.pop(0)]
            i -= 1

            # Check for tie, and process accordingly
            if sum(p1_play) > sum(p2_play):
                # Player 1 wins hand, append cards to claim list
                p1_claim.extend(p1_play); p1_claim.extend(p2_play)
                claim.append(sum(p1_play) + sum(p2_play))
                #print('\tPlayer 1 wins hand (%d over %d), collects' % (sum(p1_play), sum(p2_play)), sum(p1_play) + sum(p2_play))
            elif sum(p2_play) > sum(p1_play):
                # Player 2 wins hand, append cards
                p2_claim.extend(p1_play); p2_claim.extend(p2_play)
                claim.append(sum(p1_play) + sum(p2_play))
                #print('\tPlayer 2 wins hand (%d over %d), collects' % (sum(p2_play), sum(p1_play)), sum(p1_play) + sum(p2_play))
            else:
                tie=tie+1
                # Tie - Continue to play two cards until a player wins
                #print('\tTie:', sum(p1_play), 'vs.', sum(p2_play))
                if i > 0:
                    while sum(p1_play[-2:]) == sum(p2_play[-2:]):
                        p1_play.extend([p1.pop(0), p1.pop(0)])
                        p2_play.extend([p2.pop(0), p2.pop(0)])
                        i -= 1

                        if sum(p1_play[-2:]) > sum(p2_play[-2:]):
                            # Player 1 wins hand, append cards to claim list
                            p1_claim.extend(p1_play); p1_claim.extend(p2_play)
                            claim.append(sum(p1_play) + sum(p2_play))
                            #print('\tPlayer 1 wins hand (%d over %d), collects' % (sum(p1_play[-2:]), sum(p2_play[-2:])), sum(p1_play) + sum(p2_play))
                        elif sum(p2_play[-2:]) > sum(p1_play[-2:]):
                            # Player 2 wins hand, append cards to claim list
                            p2_claim.extend(p1_play); p2_claim.extend(p2_play)
                            claim.append(sum(p1_play) + sum(p2_play))
                            #print('\tPlayer 2 wins hand (%d over %d), collects' % (sum(p2_play[-2:]), sum(p1_play[-2:])), sum(p1_play) + sum(p2_play))
                        else:
                            #print('\t\tAnother Tie:', sum(p1_play[-2:]), 'vs.', sum(p2_play[-2:]))

                            # Check for whether game ends in a tie - return cards to respective players
                            if i <= 0:
                                p1_claim.extend(p1_play)
                                p2_claim.extend(p2_play)
                                break
                else:
                    # Game ends in a tie - return cards to respective players
                    p1_claim.extend(p1_play)
                    p2_claim.extend(p2_play)

            # Increment round
            k += 1
        
        max_claim = max(claim)
        results = {"Game No.":game, "Player 1 initial total":s1, "Player 2 initial total":s2, "Tie count":tie, "Highest round total":max_claim, "Player 1 final total":sum(p1_claim), "Player 2 final total":sum(p2_claim)}
        results_l.append(results)
        game = game + 1

        # Game over, compile results
        S['final_sums'] = (sum(p1_claim), sum(p2_claim))
        if S['final_sums'][0] != S['final_sums'][1]:
            # One player wins
            S['winner'] = 0 if S['final_sums'][0] > S['final_sums'][1] else 1
          #  print('\nPlayer %d wins:' % [1, 2][S['winner']], S['final_sums'][S['winner']], 'to', S['final_sums'][1 - S['winner']])
        else:
            # Game is tied
            S['winner'] = -1
          #  print('\nTie Game: %d to %d' % (S['final_sums'][0], S['final_sums'][1]))

        # Return results
    return results_l

### Step 1.4
Demonstrate your function for 1-game (games = 1) and 5-game samples (games = 5), and display the resultant dictionaries for each case.

In [203]:
#function for game = 1
play(1)

[{'Game No.': 1,
  'Player 1 initial total': 197,
  'Player 2 initial total': 167,
  'Tie count': 2,
  'Highest round total': 68,
  'Player 1 final total': 175,
  'Player 2 final total': 189}]

In [142]:
#function for game = 5
play(5)

[{'Game No.': 1,
  'Player 1 initial total': 195,
  'Player 2 initial total': 169,
  'Tie count': 1,
  'Highest round total': 59,
  'Player 1 final total': 237,
  'Player 2 final total': 127},
 {'Game No.': 2,
  'Player 1 initial total': 184,
  'Player 2 initial total': 180,
  'Tie count': 2,
  'Highest round total': 54,
  'Player 1 final total': 253,
  'Player 2 final total': 111},
 {'Game No.': 3,
  'Player 1 initial total': 172,
  'Player 2 initial total': 192,
  'Tie count': 0,
  'Highest round total': 37,
  'Player 1 final total': 130,
  'Player 2 final total': 234},
 {'Game No.': 4,
  'Player 1 initial total': 168,
  'Player 2 initial total': 196,
  'Tie count': 0,
  'Highest round total': 41,
  'Player 1 final total': 169,
  'Player 2 final total': 195},
 {'Game No.': 5,
  'Player 1 initial total': 175,
  'Player 2 initial total': 189,
  'Tie count': 1,
  'Highest round total': 66,
  'Player 1 final total': 154,
  'Player 2 final total': 210}]

## Question 2
Simulate 1000 games and store your results in a pandas DataFrame. Preview the first and last 10 rows of your results as proof of a successful run.

In [205]:
#1000 games
#storing results in df - data_frame
#previewing first 10 rows
import pandas as pd
play(1000)
data_frame = pd.DataFrame(results_l)
data_frame.set_index("Game No.",inplace=True)
data_frame = data_frame[["Player 1 initial total", "Player 1 final total", "Player 2 initial total", "Player 2 final total", "Tie count", "Highest round total"]]
data_frame.head(10)

,Player 1 initial total,Player 1 final total,Player 2 initial total,Player 2 final total,Tie count,Highest round total
Game No.,,,,,,
1,169,106,195,258,1,67
2,202,277,162,87,1,40
3,170,168,194,196,0,40
4,162,151,202,213,1,43
5,155,81,209,283,0,39
6,177,95,187,269,1,56
7,155,158,209,206,1,71
8,174,185,190,179,0,36
9,172,166,192,198,0,41


In [172]:
#last 10 rows 
data_frame.tail(10)

,Player 1 initial total,Player 1 final total,Player 2 initial total,Player 2 final total,Tie count,Highest round total
Game No.,,,,,,
991,180,128,184,236,0,39
992,174,135,190,229,0,42
993,197,238,167,126,0,38
994,177,113,187,251,1,57
995,179,145,185,219,0,45
996,195,219,169,145,0,41
997,187,213,177,151,1,49
998,198,229,166,135,0,44
999,170,157,194,207,0,39


## Question 3
Given your DataFrame of simulated game results, let's determine the outcome of each game, based on the final player totals.

### Step 3.1
Add a new column to your DataFrame ('outcome') that contains the value 'Player 1 wins' if Player 1's final total is higher, 'Player 2 wins' if Player 2's final total is higher, or 'Tie game' if the two player's final totals are equal.

In [206]:
#Adddng column 'outcome' to the dataframe to track winner
def create_col(outcome):
    if outcome["Player 1 final total"] > outcome["Player 2 final total"]:
        result = "Player 1 wins"
    elif outcome["Player 1 final total"] < outcome["Player 2 final total"]:
        result = "Player 2 wins"
    else:
        result = "Game tied"
    return result

data_frame["Outcome"] = data_frame.apply(create_col, axis=1)

### Step 3.2
Add new columns for the winning total ('winning_total') and losing total ('losing_total'), which contain the final totals for the winning and losing players, respectively, and missing values (NumPy.nan) for tie games. 

In [182]:
#Adding new columns - win_tot, loss_tot and missing values to track winning total, losing total and ties  
def create_col1(win_tot):
    if win_tot["Player 1 final total"] > win_tot["Player 2 final total"]:
        result1 = win_tot["Player 1 final total"]
    elif win_tot["Player 1 final total"] < win_tot["Player 2 final total"]:
        result1 = win_tot["Player 2 final total"]
    else:
        result1 = np.nan
    return result1

def create_col2(loss_tot):
    if loss_tot["Player 1 final total"] > loss_tot["Player 2 final total"]:
        result2 = loss_tot["Player 2 final total"]
    elif loss_tot["Player 1 final total"] < loss_tot["Player 2 final total"]:
        result2 = loss_tot["Player 1 final total"]
    else:
        result2 = np.nan
    return result2

data_frame["Winning total"] = data_frame.apply(create_col1,axis=1)
data_frame["Losing total"] = data_frame.apply(create_col2,axis=1)

### Step 3.3
Preview the first 10 rows of your modified DataFrame. Then, summarize the frequency of the three outcomes and display the rows in your DataFrame that contain the 'Tie game' outcomes.

In [183]:
#Previewing first 10 rows
data_frame.head(10)

,Player 1 initial total,Player 1 final total,Player 2 initial total,Player 2 final total,Tie count,Highest round total,Outcome,Winning total,Losing total
Game No.,,,,,,,,,
1,179,134,185,230,0,39,Player 2 wins,230.0,134.0
2,168,141,196,223,0,45,Player 2 wins,223.0,141.0
3,179,175,185,189,1,37,Player 2 wins,189.0,175.0
4,198,239,166,125,0,42,Player 1 wins,239.0,125.0
5,155,103,209,261,0,43,Player 2 wins,261.0,103.0
6,188,177,176,187,1,55,Player 2 wins,187.0,177.0
7,178,212,186,152,1,48,Player 1 wins,212.0,152.0
8,208,205,156,159,0,38,Player 1 wins,205.0,159.0
9,197,263,167,101,0,37,Player 1 wins,263.0,101.0


In [186]:
#Summarising frequencies of the 3 outcomes
print(data_frame["Outcome"].value_counts())
data_frame.loc[data_frame["Outcome"] == "Game tied"]

Player 2 wins    497
Player 1 wins    495
Game tied          8
Name: Outcome, dtype: int64


,Player 1 initial total,Player 1 final total,Player 2 initial total,Player 2 final total,Tie count,Highest round total,Outcome,Winning total,Losing total
Game No.,,,,,,,,,
215,188,182,176,182,0,41,Game tied,NaN,NaN
305,174,182,190,182,0,37,Game tied,NaN,NaN
345,185,182,179,182,0,39,Game tied,NaN,NaN
409,194,182,170,182,1,44,Game tied,NaN,NaN
583,168,182,196,182,1,62,Game tied,NaN,NaN
675,179,182,185,182,0,38,Game tied,NaN,NaN
807,173,182,191,182,1,38,Game tied,NaN,NaN
819,178,182,186,182,0,39,Game tied,NaN,NaN


### Step 3.4
Produce descriptive statistics (e.g., minimum, mean, standard deviation, maximum) for your winning total column, and display the rows in your DataFrame that contain the top 10 winning totals. 

In [188]:
#Descriptive statistics for 'Winning total'
#Displaying rows with top 10 winning totals
print(data_frame["Winning total"].describe())
sorted_data_frame=data_frame.sort_values(by="Winning total", ascending=False)
sorted_data_frame.head(10)

count    992.000000
mean     226.203629
std       31.236200
min      183.000000
25%      202.000000
50%      221.000000
75%      244.250000
max      349.000000
Name: Winning total, dtype: float64


,Player 1 initial total,Player 1 final total,Player 2 initial total,Player 2 final total,Tie count,Highest round total,Outcome,Winning total,Losing total
Game No.,,,,,,,,,
823,147,15,217,349,1,57,Player 2 wins,349.0,15.0
432,149,29,215,335,1,50,Player 2 wins,335.0,29.0
592,156,33,208,331,0,41,Player 2 wins,331.0,33.0
874,209,329,155,35,1,74,Player 1 wins,329.0,35.0
842,156,35,208,329,0,42,Player 2 wins,329.0,35.0
468,207,320,157,44,1,51,Player 1 wins,320.0,44.0
489,149,49,215,315,1,60,Player 2 wins,315.0,49.0
487,199,314,165,50,1,54,Player 1 wins,314.0,50.0
304,164,50,200,314,2,66,Player 2 wins,314.0,50.0


## Question 4

### Step 4.1
Given your DataFrame of simulated game results, calculate the frequency of the number of ties within each game (i.e., the number of rounds in which a tie occurs).

In [191]:
#frequency of number of ties within each game
data_frame["Tie count"].value_counts()

0    517
1    369
2     94
3     19
4      1
Name: Tie count, dtype: int64

### Step 4.2
Calculate the mean maximum round total as a function of the number of ties. What do you observe about the relationship between the number of ties and the mean maximum round total?

In [199]:
#calculating mean max round totals for different tie values 
mean0 = data_frame.loc[data_frame["Tie count"] == 0, "Highest round total"].sum()/data_frame.loc[data_frame["Tie count"] == 0, "Tie count"].count()
print("Mean when 0 ties-",mean0)

mean1 = data_frame.loc[data_frame["Tie count"] == 1, "Highest round total"].sum()/data_frame.loc[data_frame["Tie count"] == 1, "Tie count"].count()
print("Mean when 1 tie-",mean1)

mean2 = data_frame.loc[data_frame["Tie count"] == 2, "Highest round total"].sum()/data_frame.loc[data_frame["Tie count"] == 2, "Tie count"].count()
print("Mean when 2 ties-",mean2)

mean3 = data_frame.loc[data_frame["Tie count"] == 3, "Highest round total"].sum()/data_frame.loc[data_frame["Tie count"] == 3, "Tie count"].count()
print("Mean when 3 ties-",mean3)

mean4 = data_frame.loc[data_frame["Tie count"] == 4, "Highest round total"].sum()/data_frame.loc[data_frame["Tie count"] == 4, "Tie count"].count()
print("Mean when 4 ties-",mean4)

Mean when 0 ties- 40.26112185686654
Mean when 1 tie- 55.36585365853659
Mean when 2 ties- 63.776595744680854
Mean when 3 ties- 67.89473684210526
Mean when 4 ties- 55.0


### Mean highest round total in games increase as the number of ties increase.